In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [15]:
imgs = np.load('/content/drive/MyDrive/images.npy')
lbls = np.load('/content/drive/MyDrive/labels.npy')


In [16]:
def process_lbls(labels,partition):
  index = 0
  mul = 60 / partition
  for lbl in labels:
    i = int(lbl[1]/partition)
    labels[index] = lbl[0] * mul + i
    index+=1
  return labels
lbls = process_lbls(lbls,1)
lbls = lbls[:,1]/1.0

In [17]:
imgs_train,imgs_test,lbls_train,lbls_test = train_test_split(imgs, lbls, train_size=0.8, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(imgs_train, lbls_train, test_size=0.125, random_state=23)

In [18]:
def create_model():
  model = keras.models.Sequential([
    keras.layers.Input(shape=(150, 150, 1)),
    keras.layers.Conv2D(32, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.Conv2D(32, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"),
    keras.layers.Conv2D(64, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.Conv2D(64, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"),
    keras.layers.Conv2D(128, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.Conv2D(128, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"),
    keras.layers.Conv2D(256, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.Conv2D(256, kernel_size=3, strides = 2, activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"),    
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(units=720, activation='softmax'),
  ])
  return model

In [7]:
def Mean_loss(y_true, y_pred): # common sense error
   cse = tf.reduce_mean(tf.math.minimum(
          tf.math.abs(y_true - y_pred),
          tf.math.abs(tf.math.minimum(y_true, y_pred) + 12 - tf.math.maximum(y_true, y_pred))))*60
   return cse

In [19]:
model = create_model()
model.compile(loss= "sparse_categorical_crossentropy",
            optimizer="sgd",
            metrics=['accuracy', Mean_loss]
            )

In [ ]:
plot_model(model)

In [ ]:
history = model.fit(X_train, y_train, epochs=400,
 validation_data=(X_valid, y_valid))

In [ ]:
X_new = imgs_test[:3]
y_proba = model.predict(X_new)
print(y_proba.round(2))